## Study session 12 - data analysis
### BIOINF 575 - Fall 2020
SOLUTION

____

### <font color = "red">Exercise</font>

[ClinVar][1] is a freely accessible, public archive of reports of the relationships among human variations and phenotypes, with supporting evidence.


The data you will be working with (`clinvar.vcf`).  More information about the database can be found [here][3].

From: https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/

Download the file clinvar.vcf.gz


### The file format
The beginning of every VCF file contains various sets of information:
* Meta-information (details about the experiment or configuration) lines start with **`##`**
    * These lines are helpful in understanding specialized keys found in the `INFO` column. It is in these sections that one can find:
        * The description of the key
        * The data type of the values
        * The default value of the values
* Header lines (column names) start with **`#`**

From there on, each line is made up of tab (`\t`) separated values that make up eight (8) columns. Those columns are:
1. CHROM (chromosome)
2. POS (base pair position of the variant)
3. ID (identifier if applicable; `.` if not applicable/missing)
4. REF (reference base)
5. ALT (alternate base(s): comma (`,`) separated if applicable)
6. QUAL (Phred-scaled quality score; `.` if not applicable/missing)
7. FILTER (filter status; `.` if not applicable/missing)
8. INFO (any additional information about the variant)
    * Semi-colon (`;`) separated key-value pairs
    * Key-value pairs are equal sign (`=`) separated (key on the left, value on the right)
    * If a key has multiple values, the values are pipe (`|`) separated

---
[1]: https://www.ncbi.nlm.nih.gov/clinvar/intro/
[2]: https://samtools.github.io/hts-specs/VCFv4.3.pdf
[3]: http://exac.broadinstitute.org




### Create a database clinvar_database.sqlite and in that database create a table `variant` and a table `variant_info` 

#### variant table columns:

- variant_id - int primary key autoincrement
- CHROM - string
- POS - int
- ID - string 
- REF - string
- ALT - string  
- QUAL - string 
- FILTER - string  

#### variant_info table columns

- variant_id int foreign key to variant_id in the variant table
- key - string
- value - string



##### WHILE DEVELOPING YOUR SOLUTION YOU MAY WANT OT RESET/CLEAR YOUR TABLES
##### you will get erros if you want to insert data that is already in the table
##### to clear a talbe use 
```sql
DELETE FROM table_name 
```

In [1]:
from sqlite3 import connect

connection = connect("clinvar_database.sqlite")
cursor = connection.cursor()

In [2]:
# nice display of results header 

def get_header(cursor):
    '''
    Makes a tab delimited header row from the cursor description.
    Arguments:
        cursor: a cursor after a select query
    Returns:
        string: A string consisting of the column names separated by tabs, no new line
    '''
    return '\t'.join([row[0] for row in cursor.description])


In [3]:
# nice display of results  

def get_results(cursor):
    '''
    Makes a tab delimited table from the cursor results.
    Arguments:
        cursor: a cursor after a select query
    Returns:
        string: A string consisting of the column names separated by tabs, no new line
    ''' 
    res = list()
    for row in cursor.fetchall():        
        res.append('\t'.join(list(map(str,row))))
    return "\n".join(res)

In [4]:
sql='''
CREATE TABLE IF NOT EXISTS variant (
 variant_id INTEGER PRIMARY KEY AUTOINCREMENT,
 CHROM  TEXT NOT NULL,
 POS  INTEGER NOT NULL, 
 ID  TEXT,
 REF  TEXT,
 ALT  TEXT,  
 QUAL TEXT, 
 FILTER  TEXT 
);
'''
try:
    cursor.execute(sql)
except connection.DatabaseError:
    print("Creating the variant table resulted in a database error!")
    connection.rollback()
    raise
else:
    connection.commit()
    print("Create table successful.")
finally:
    print("done!")

Create table successful.
done!


In [5]:
sql='''
CREATE TABLE IF NOT EXISTS variant_info (
 variant_id INTEGER NOT NULL,
 key  TEXT NOT NULL,
 value TEXT NOT NULL,
 FOREIGN KEY (variant_id) REFERENCES  variant  (variant_id)
);
'''
try:
    cursor.execute(sql)
except connection.DatabaseError:
    print("Creating the variant_info table resulted in a database error!")
    connection.rollback()
    raise
else:
    connection.commit()
    print("Create table successful.")
finally:
    print("done!")

Create table successful.
done!


In [6]:
# check in the sqlite_master table that we have the new tables
select_count_variant = """
SELECT type, name 
FROM sqlite_master;
"""
cursor.execute(select_count_variant)

print(get_header(cursor))
print(get_results(cursor))


type	name
table	variant
table	sqlite_sequence
table	variant_info


### Populate the `variant` table and the `variant_info` table with the information form clinvar.vcf



In [7]:
variant_file_name = "clinvar.vcf"


In [8]:
# WHILE DEVELOPING YOUR SOLUTION YOU MAY WANT OT RESET/CLEAR YOUR TABLES
# you will get erros if you want to insert data that is already in the table

cursor.execute("DELETE FROM variant_info;")
cursor.execute("DELETE FROM variant;")
connection.commit()

In [9]:
### Version 1 insert elements one by one as we build them from the file
### use cursor.execute and a list with the elements for one row in the file

variant_id = 0
variant_list = []
info_list = []
with open(variant_file_name) as var_file:
    for line in var_file:
        if not line.startswith('#'):
            variant_id +=1
            *variant_list_elem, info_text = line.strip().split("\t")
            variant_list_elem.insert(0,variant_id)
            #print(variant_list_elem)
            insert_statement_var = f"""
                INSERT INTO variant (variant_id, CHROM, POS, ID, REF, ALT, QUAL, FILTER)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_statement_var, variant_list_elem)

            for info_pair in info_text.split(";"):
                key,values = info_pair.split("=")
                for value in values.split("|"):
                    info_elem = (variant_id, key, value)
                    #print(info_elem)
                    insert_statement_var_info = f"""
                        INSERT INTO variant_info (variant_id, key, value)
                        VALUES (?, ?, ?);
                    """
                    cursor.execute(insert_statement_var_info, info_elem)
            #break
            if variant_id % 100000 == 0:
                print("processed",variant_id, "rows")
            
connection.commit()

processed 100000 rows
processed 200000 rows
processed 300000 rows
processed 400000 rows
processed 500000 rows
processed 600000 rows
processed 700000 rows


_____

In [10]:
# WHILE DEVELOPING YOUR SOLUTION YOU MAY WANT OT RESET/CLEAR YOUR TABLES
# you will get erros if you want to insert data that is already in the table

cursor.execute("DELETE FROM variant_info;")
cursor.execute("DELETE FROM variant;")
connection.commit()

In [11]:
### Version 1.1 insert elements one by one as we build them from the file
### use cursor.execute and a list with the elements for one row in the file
### use cursor.lastrowid to get the variant id

variant_id = 0
variant_list = []
info_list = []
with open(variant_file_name) as var_file:
    for line in var_file:
        if not line.startswith('#'):
            variant_id +=1
            *variant_list_elem, info_text = line.strip().split("\t")

            #print(variant_list_elem)
            insert_statement_var = f"""
                INSERT INTO variant (CHROM, POS, ID, REF, ALT, QUAL, FILTER)
                VALUES (?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_statement_var, variant_list_elem)
            variant_id = cursor.lastrowid
            
            for info_pair in info_text.split(";"):
                key,values = info_pair.split("=")
                for value in values.split("|"):
                    info_elem = (variant_id, key, value)
                    #print(info_elem)
                    insert_statement_var_info = f"""
                        INSERT INTO variant_info (variant_id, key, value)
                        VALUES (?, ?, ?);
                    """
                    cursor.execute(insert_statement_var_info, info_elem)
            #break
            if variant_id % 100000 == 0:
                print("processed",variant_id, "rows")
            
connection.commit()

processed 800000 rows
processed 900000 rows
processed 1000000 rows
processed 1100000 rows
processed 1200000 rows
processed 1300000 rows
processed 1400000 rows
processed 1500000 rows


______

In [12]:
# WHILE DEVELOPING YOUR SOLUTION YOU MAY WANT OT RESET/CLEAR YOUR TABLES
# you will get erros if you want to insert data that is already in the table

cursor.execute("DELETE FROM variant_info;")
cursor.execute("DELETE FROM variant;")
connection.commit()

In [13]:
# THIS IS part of version 2 of the solution.

# Function to create a template for sql insert command 
# and return the command text for a specific table

def insert_function(table_name,col_names):
    
    insert_template = f"""
    INSERT INTO {table_name} ({",".join(col_names)})
    VALUES ({",".join(["?" for i in range(len(col_names))])});
    """
    
    return insert_template

In [14]:
### Version 2 insert all elements at the same time after we build list of elements (rows) for each table
### use cursor.executemany and a list of lists wheter each element of the big list is 
### a list/tuple with the elements for one row in the file


variant_id = 0
variant_list = []
info_list = []
with open(variant_file_name) as var_file:
    for line in var_file:
        if not line.startswith('#'):
            variant_id +=1
            *variant_list_elem, info_text = line.strip().split("\t")
            variant_list_elem.insert(0,variant_id)
            variant_list.append(variant_list_elem)
            #print(variant_list)
            for info_pair in info_text.split(";"):
                key,values = info_pair.split("=")
                for value in values.split("|"):
                    info_elem = (variant_id, key, value)
                    info_list.append(info_elem)
            #print(info_list)
            #break
            if variant_id % 100000 == 0:
                print("processed",variant_id, "rows")
                
variant_cols = ("variant_id", "CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER") 
variant_info_cols = ("variant_id", "key", "value")

insert_statement = insert_function("variant", variant_cols)
cursor.executemany(insert_statement, variant_list)

insert_statement = insert_function("variant_info", variant_info_cols)
cursor.executemany(insert_statement, info_list)

connection.commit()

processed 100000 rows
processed 200000 rows
processed 300000 rows
processed 400000 rows
processed 500000 rows
processed 600000 rows
processed 700000 rows


______

### Use a select statment to retrieve the number of rows in the variats table



In [15]:
select_count_variant = """
SELECT count(*) "VARIANT NUMBER"
FROM variant;
"""
cursor.execute(select_count_variant)

print(get_header(cursor))
print(get_results(cursor))

VARIANT NUMBER
778088


### Use a select statment to compute the number of info elements for each variant
#### Retrieve the first 20 rows


In [16]:
select_count_variant = """
SELECT variant_id VAR_ID, count(variant_id) AS INFO_NO
FROM variant_info
GROUP BY variant_id
LIMIT 20;
"""
cursor.execute(select_count_variant)

print(get_header(cursor))
print(get_results(cursor))

VAR_ID	INFO_NO
1	12
2	12
3	16
4	12
5	12
6	12
7	12
8	12
9	12
10	12
11	12
12	12
13	12
14	12
15	12
16	12
17	13
18	12
19	12
20	12


### Use a select statment to retrieve the variant identifier (ID column in the variant table should be different than ".") and gene names (key = GENEINFO) in the varinat_info table for variants in chromosome 3
#### Limit result to  to 50 rows.

In [17]:
select_count_variant = """
SELECT v.ID VID, vi.value GENE_INFO
FROM variant v
JOIN variant_info vi ON v.variant_id = vi.variant_id
WHERE v.CHROM = '3' AND v.ID <> "." AND vi.key = "GENEINFO"
LIMIT 50;
"""
cursor.execute(select_count_variant)

print(get_header(cursor))
print(get_results(cursor))

VID	GENE_INFO
983061	CHL1:10752
718047	CHL1:10752
758185	CHL1:10752
767882	CHL1:10752
773668	CHL1:10752
738430	CHL1:10752
726676	CHL1:10752
724082	CHL1:10752
708287	CHL1:10752
787658	CHL1:10752
748438	CHL1:10752
739136	CHL1:10752
750976	CHL1:10752
783995	CHL1:10752
787251	CHL1:10752
708386	CHL1:10752
763543	CHL1:10752
379251	CHL1:10752
750399	CHL1:10752
718007	CHL1:10752
731515	CHL1:10752
716766	CHL1:10752
789576	CHL1:10752
740758	CHL1:10752
740758	CHL1-AS1:101927193
710904	CHL1:10752
710904	CHL1-AS1:101927193
743671	CHL1:10752
743671	CHL1-AS1:101927193
729796	CHL1:10752
729796	CHL1-AS1:101927193
799385	CHL1:10752
799385	CHL1-AS1:101927193
788967	CHL1:10752
731691	CHL1:10752
730538	CHL1:10752
782983	CHL1:10752
791904	CHL1:10752
790345	CHL1:10752
754934	CHL1:10752
792392	CHL1:10752
757696	CHL1:10752
737303	CHL1:10752
718247	CHL1:10752
728649	CHL1:10752
746696	CHL1:10752
773864	CNTN6:27255
774840	CNTN6:27255
758226	CNTN6:27255
778061	CNTN6:27255


In [18]:
# Select variants that have an RS identifier (value for RS key in variant_info)
# (id in dbSNP, https://www.ncbi.nlm.nih.gov/snp/) and 
# a gene associated with it (value for GENEINFO key in variant_info)
# The first result is related to the variant described at:
# https://www.ncbi.nlm.nih.gov/snp/rs139673243
    
select_count_variant = """
SELECT v.ID VID, v.POS, vi.value RSID, vi1.value GENE_INFO
FROM variant v
JOIN variant_info vi ON v.variant_id = vi.variant_id
JOIN variant_info vi1 ON v.variant_id = vi1.variant_id
WHERE v.CHROM = '3' AND v.ID <> "." AND vi.key = "RS" AND vi1.key = "GENEINFO"
LIMIT 10;
"""
cursor.execute(select_count_variant)

print(get_header(cursor))
print(get_results(cursor))

VID	POS	RSID	GENE_INFO
718047	319804	139673243	CHL1:10752
758185	319812	766975263	CHL1:10752
767882	319864	116261368	CHL1:10752
773668	325956	200543338	CHL1:10752
738430	328173	200045243	CHL1:10752
726676	328209	1373131590	CHL1:10752
724082	328248	13318299	CHL1:10752
708287	328363	780875423	CHL1:10752
787658	340846	148439738	CHL1:10752
748438	341925	143704896	CHL1:10752
